This notebook computes a monthly climatology in several different ways to experiment with the efficiency of different options. It uses OISST as an example dataset.

### Imports and dataset load

In [1]:
import os

import s3fs
import xarray as xr
import dask

from dask.distributed import Client

In [2]:
TEMP_DIR = os.getenv('TEMP_DIR')

dask.config.set(temporary_directory=TEMP_DIR)
client = Client(memory_limit='216GB')

/homes/metogra/rwegener/.conda/envs/mhw/lib/python3.8/site-packages/distributed/node.py:160: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 38180 instead
  warnings.warn(


In [3]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:38180/status,
Dashboard: http://127.0.0.1:38180/status,Workers: 16
Total threads: 128,Total memory: 3.14 TiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:38835,Workers: 16
Dashboard: http://127.0.0.1:38180/status,Total threads: 128
Started: Just now,Total memory: 3.14 TiB
Comm: tcp://127.0.0.1:38984,Total threads: 8
Dashboard: http://127.0.0.1:37073/status,Memory: 201.17 GiB
Nanny: tcp://127.0.0.1:37192,


In [4]:
endpoint_url = 'https://ncsa.osn.xsede.org'
fs_osn = s3fs.S3FileSystem(anon=True, client_kwargs={'endpoint_url': endpoint_url},) 

path = "Pangeo/pangeo-forge/noaa_oisst/v2.1-avhrr.zarr"
ds = xr.open_zarr(fs_osn.get_mapper(path), consolidated=True)
oisst = ds.sst.isel(zlev=0).drop('zlev')  # (14532, 720, 1440)

In [5]:
# Roughly 5 years of data
oisst_subset = oisst.isel(time=slice(1800))

In [6]:
oisst_subset

<xarray.DataArray 'sst' (time: 1800, lat: 720, lon: 1440)>
dask.array<getitem, shape=(1800, 720, 1440), dtype=float32, chunksize=(20, 720, 1440), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float32 -89.88 -89.62 -89.38 -89.12 ... 89.38 89.62 89.88
  * lon      (lon) float32 0.125 0.375 0.625 0.875 ... 359.1 359.4 359.6 359.9
  * time     (time) datetime64[ns] 1981-09-01T12:00:00 ... 1986-08-05T12:00:00
Attributes:
    long_name:  Daily sea surface temperature
    units:      Celsius
    valid_max:  4500
    valid_min:  -300

### Approach 1: Regular groupby

- 5 year subset: ~1900 tasks, ~4 MB chunks, ~4 min runtime

In [7]:
oisst_subset.groupby('time.month').mean(dim='time')

<xarray.DataArray 'sst' (month: 12, lat: 720, lon: 1440)>
dask.array<stack, shape=(12, 720, 1440), dtype=float32, chunksize=(1, 720, 1440), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float32 -89.88 -89.62 -89.38 -89.12 ... 89.38 89.62 89.88
  * lon      (lon) float32 0.125 0.375 0.625 0.875 ... 359.1 359.4 359.6 359.9
  * month    (month) int64 1 2 3 4 5 6 7 8 9 10 11 12

In [8]:
%%time

climatology = oisst.groupby('time.month').mean(dim='time').compute()

CPU times: user 23.7 s, sys: 3.05 s, total: 26.7 s
Wall time: 4min 7s


In [ ]:
climatology

### Approach 2: flox map-reduce

~3900 tasks, ~47 MB chunks, ~36 seconds (!) runtime

In [9]:
from flox.xarray import xarray_reduce

In [12]:
clim_flox = xarray_reduce(oisst_subset, 'time.month', func='mean', method='map-reduce')

clim_flox

<xarray.DataArray 'sst' (month: 12, lat: 720, lon: 1440)>
dask.array<transpose, shape=(12, 720, 1440), dtype=float32, chunksize=(12, 720, 1440), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float32 -89.88 -89.62 -89.38 -89.12 ... 89.38 89.62 89.88
  * lon      (lon) float32 0.125 0.375 0.625 0.875 ... 359.1 359.4 359.6 359.9
  * month    (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
Attributes:
    long_name:  Daily sea surface temperature
    units:      Celsius
    valid_max:  4500
    valid_min:  -300

In [13]:
%time clim_flox.load()

CPU times: user 3.33 s, sys: 577 ms, total: 3.9 s
Wall time: 36.1 s


<xarray.DataArray 'sst' (month: 12, lat: 720, lon: 1440)>
array([[[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        ...,
        [-1.69548379, -1.69548379, -1.69548379, ..., -1.69561275,
         -1.69548379, -1.69548379],
        [-1.69812878, -1.6980644 , -1.69799982, ..., -1.69825794,
         -1.69812878, -1.69812878],
        [-1.6993548 , -1.6993548 , -1.6993548 , ..., -1.69941918,
         -1.69941918, -1.6993548 ]],

       [[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
...
        [-1.63440002, -1.63440002, -1.63440002, ..., -1.63446655,
         -1.63446655, -1.63446655],
        [-1.63606659, -1.63613332, -1.63613332, ..., -1.63619995,
         -1.63619985, -1.63613332],
        [-1.63726664, -1.63726664, -1.63726664, ..., -1.63733337,
         -1.63726664, -1.63726664]],

       [[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        ...,
        [-1.67754832, -1.67754832, -1.67741935, ..., -1.6776129 ,
         -1.6776129 , -1.67754832],
        [-1.68025808, -1.68025808, -1.68025808, ..., -1.68025808,
         -1.68025808, -1.68025808],
        [-1.6814839 , -1.6814839 , -1.6814839 , ..., -1.6814839 ,
         -1.6814839 , -1.6814839 ]]])
Coordinates:
  * lat      (lat) float32 -89.88 -89.62 -89.38 -89.12 ... 89.38 89.62 89.88
  * lon      (lon) float32 0.125 0.375 0.625 0.875 ... 359.1 359.4 359.6 359.9
  * month    (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
Attributes:
    long_name:  Daily sea surface temperature
    units:      Celsius
    valid_max:  4500
    valid_min:  -300

### Approach 3: map_blocks

In [ ]:
def calculate_clim(chunk):
    return chunk.groupby('time.month').mean(dim='time')

In [ ]:
xr.map_blocks(calculate_clim, oisst_subset)

### Repeat?

If I rechunk this on load and re-try it do I get different results?